In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from allora_sdk import AlloraClient
from datetime import datetime, timedelta
import asyncio

# Initialize Allora client
client = AlloraClient(
    chain="testnet",
    api_key="API_KEY"
)

In [ ]:
# Function to fetch historical predictions from Allora
async def fetch_historical_predictions(start_date, end_date, interval_hours=8):
    predictions = []
    current_date = start_date
    
    while current_date <= end_date:
        # Get BTC and ETH predictions for each time period
        btc_pred = client.get_price_inference(
            asset="BTC",
            timeframe="8h"
        )
        eth_pred = client.get_price_inference(
            asset="ETH",
            timeframe="8h"
        )
        
        predictions.append({
            'timestamp': current_date,
            'btc_prediction': btc_pred['score'],
            'eth_prediction': eth_pred['score'],
            'risk_topics': [t for t in client.get_all_topics() if t.get('category') == 'risk']
        })
        
        current_date += timedelta(hours=interval_hours)
    
    return pd.DataFrame(predictions)

In [ ]:
# Backtesting strategy using Allora's predictions
def backtest_strategy(data, initial_capital=10000):
    portfolio_value = initial_capital
    positions = []
    
    for idx, row in data.iterrows():
        # Strategy logic based on Allora predictions
        yield_score = (row['btc_prediction'] + row['eth_prediction']) / 2
        risk_score = len(row['risk_topics']) / 10  # Simplified risk calculation
        
        # Execute trade if good opportunity
        if yield_score > 0.1 and risk_score < 0.5:  # If good yield and low risk
            position_size = portfolio_value * 0.1    # Use 10% of portfolio
            estimated_return = yield_score * position_size
            portfolio_value += estimated_return
            
            positions.append({
                'timestamp': row['timestamp'],
                'action': 'TRADE',
                'portfolio_value': portfolio_value,
                'yield_score': yield_score,
                'risk_score': risk_score
            })
    
    return pd.DataFrame(positions)

In [ ]:
# Run backtest for last 30 days
async def run_backtest():
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30)
    
    # Get the data
    print("Fetching historical predictions...")
    historical_data = await fetch_historical_predictions(start_date, end_date)
    
    # Test our strategy
    print("Running backtest...")
    results = backtest_strategy(historical_data)
    
    # Create nice graphs
    plt.figure(figsize=(12, 6))
    plt.plot(results['timestamp'], results['portfolio_value'])
    plt.title('Strategy Performance (Using Allora Predictions)')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value')
    plt.grid(True)
    
    plt.figure(figsize=(12, 4))
    plt.plot(results['timestamp'], results['yield_score'], label='Yield Score')
    plt.plot(results['timestamp'], results['risk_score'], label='Risk Score')
    plt.title('Yield and Risk Scores Over Time')
    plt.xlabel('Date')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return results

In [ ]:
# Run the backtest
results = await run_backtest()
print("\nBacktest Summary:")
print(f"Total trades: {len(results)}")
print(f"Final portfolio value: ${results['portfolio_value'].iloc[-1]:.2f}")
print(f"Average yield score: {results['yield_score'].mean():.3f}")
print(f"Average risk score: {results['risk_score'].mean():.3f}")